Code to extract numerical and time series data.

Things to note:

Because the chartevents data is very large only 20,000,000 rows were loaded

In [1]:
# Import libraries
import os
import pandas as pd

In [2]:
# Load the data

#items 
items = pd.read_csv(os.path.join('..','d_items.csv'), usecols= ['itemid', 'label', 'abbreviation'])

items = items[items['itemid'].isin([220210, 220277, 225309, 220045, 220739, 223900, 223901, 223762])]


Option 1: Using the earliest chart time for each hadm_id


In [3]:
# chartevents
chart = pd.read_csv(os.path.join('..','chartevents.csv'), usecols= ['subject_id', 'hadm_id', 'charttime','itemid', 'valuenum', 'valueuom'], nrows=20000000)

chart = chart[chart['itemid'].isin([220210, 220277, 225309, 220045, 220739, 223900, 223901, 223762])]

chart['charttime'] = pd.to_datetime(chart['charttime'])

chart = chart.sort_values(by = ['hadm_id', 'charttime'] )

chart = chart.groupby('hadm_id').first().reset_index()


In [4]:
chart.shape

(4351, 6)

In [5]:
# Combining the dataset
num_data = pd.merge(chart, items, on = ['itemid'])

In [6]:
# Use the items as columns
num_data = num_data.pivot_table(index= ['subject_id', 'hadm_id', 'charttime'], columns = 'label', values = 'valuenum').reset_index()

In [7]:
# Combine all the GCS to get the total
num_data['GCS Total'] = num_data[['GCS - Eye Opening', 'GCS - Motor Response', 'GCS - Verbal Response']].sum(axis=1)

num_data = num_data.drop(columns = ['GCS - Eye Opening', 'GCS - Motor Response', 'GCS - Verbal Response'])

In [8]:
# Get the age column
patient_data = pd.read_csv(os.path.join('..','patients.csv', 'patients.csv'), usecols= ['subject_id','anchor_age'])

In [9]:
# Merge the age column to the numerical dataset
final = pd.merge(patient_data, num_data, on = ['subject_id'])

In [10]:
final.to_csv('NumericalData.csv')